In [1]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
import evaluate
import numpy as np

In [ ]:
summarizer = pipeline('summarization')
text = '''Text mining, also referred to as text data mining (abbr.: TDM), similar to text analytics, 
        is the process of deriving high-quality information from text. It involves 
        "the discovery by computer of new, previously unknown information, 
        by automatically extracting information from different written resources." 
        Written resources may include websites, books, emails, reviews, and articles. 
        High-quality information is typically obtained by devising patterns and trends 
        by means such as statistical pattern learning. According to Hotho et al. (2005)
        we can distinguish between three different perspectives of text mining: 
        information extraction, data mining, and a KDD (Knowledge Discovery in Databases) process.''' 
result = summarizer(text)
result[0]['summary_text']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-small', model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
type(tokenizer), type(model)

In [ ]:
preprocess_text = text.strip().replace('\n', '')
input_text = 'summarize: ' + preprocess_text
tokenized_text = tokenizer.encode(input_text, return_tensors='pt').to(device)
summary_ids = model.generate(tokenized_text, num_beams=4, no_repeat_ngram_size=3,
                             min_length=30, max_length=200, early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(output)

In [ ]:
tokenized_text, summary_ids

In [ ]:
input_text = 'translate english to german: That is good'
tokenized_text = tokenizer.encode(input_text, return_tensors='pt').to(device)
summary_ids = model.generate(tokenized_text, num_beams=4, no_repeat_ngram_size=3,
                             max_length=200, early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5TokenizerFast.from_pretrained('t5-small', model_max_length=1024)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
text = '''The Inflation Reduction Act lowers prescription drug costs, health care costs, 
and energy costs. It's the most aggressive action on tackling the climate crisis in American history, 
which will lift up American workers and create good-paying, union jobs across the country. 
It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. 
And no one making under $400,000 per year will pay a penny more in taxes.'''

In [ ]:
preprocess_text = text.strip().replace('\n', '')
input_text = 'summarize: ' + preprocess_text

In [ ]:
tokenized_text = tokenizer.encode(input_text, return_tensors='pt').to(device)
summary_ids = model.generate(tokenized_text,
                             num_beams=4, no_repeat_ngram_size=3,
                             min_length=30, max_length=100, early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
output

In [ ]:
billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.2)
example = billsum["train"][0]
for key in example.keys():
    print(key, '\n', example[key])
    print('\n\n')

In [ ]:
def preprocess_text(data):
    inputs = ["summarize: " + doc for doc in data["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(data["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_text, batched=True, remove_columns=billsum["train"].column_names)
tokenized_billsum

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
)
trainer = Seq2SeqTrainer(
    tokenizer=tokenizer,
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
summary_ids = model.generate(tokenized_text,
                             num_beams=4, no_repeat_ngram_size=3,
                             min_length=30, max_length=100, early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
output

In [ ]:
text = """디아블로는 액션 롤플레잉 핵 앤드 슬래시 비디오 게임이다. 
플레이어는 주변 환경을 마우스로 사용해 영웅을 움직이게 한다. 
주문을 외는 등의 다른 활동은 키보드 입력으로 이루어진다. 
플레이어는 이 게임에서 장비를 획득하고, 주문을 배우고, 적을 쓰러뜨리며, NPC와 대화를 나눌 수 있다.
지하 미궁은 주어진 형식이 있고 부분적으로 반복되는 형태가 존재하나 전체적으로 보면 무작위로 생성된다. 
예를 들어 지하 묘지의 경우에는 긴 복도와 닫힌 문들이 존재하고, 동굴은 좀 더 선형 형태를 띠고 있다. 
플레이어에게는 몇몇 단계에서 무작위의 퀘스트를 받는다. 
이 퀘스트는 선택적인 사항이나 플레이어의 영웅들을 성장시키거나 줄거리를 이해하는데 도움을 준다. 
그러나 맨 뒤에 두 퀘스트는 게임을 끝내기 위해 완료시켜야 한다."""

preprocess_text = text.strip().replace("\n", "")

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt")
summary_ids = model.generate(tokenized_text,
                             num_beams=4, no_repeat_ngram_size=3,
                             min_length=10, max_length=150, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)